In [1]:
import os 

# The normal imports
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', -1)

# Import the stats library
from scipy import stats

# These are the plottinglibraries we'll use:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import seaborn as sns

# Command for plots to appear in the iPython Notebook
%matplotlib inline

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

#Starting with the dataset that Dr.Drew helped clean.->highschools saved to .cvs file from graduations rates (2) notebook
wd = os.getcwd() #get working directory
highschools = pd.read_csv(wd+'\\data\\highschools.csv', low_memory=False)
highschools.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Columns: 294 entries, Unnamed: 0 to Percent GLP
dtypes: bool(8), float64(264), int64(9), object(13)
memory usage: 1.0+ MB


#### Business Understanding [10]

Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). Describe how you would define and measure the outcomes from the dataset. That is, why is this data important and how do you know if you have mined useful knowledge from the dataset? 
How would you measure the effectiveness of a good prediction algorithm? Be specific.

#### Data Meaning Type  [10]

Describe the meaning and type of data (scale, values, etc.) for each attribute in the data file.

#### Data Quality [15]

Verify data quality: Explain any missing values, duplicate data, and outliers. Are those mistakes? How do you deal with these problems? Give justifications for your methods.

In [4]:
#Get dimensions of the dataframe that we working with
highschools.shape

(478, 294)

In [5]:
#Make a copy of the dataset to work with
HighschoolData = highschools.copy()

##### *Missing Data Statistics*

In this section we explore missing data in the highschool data that was created from the original north carolina data. After exploring the missing data we will discusss the approach for handling them.

In [18]:
#Check for missing values
Temp = pd.DataFrame(HighschoolData.isnull().sum())
#print(Temp)
Temp.columns =['DataMissing']
#Columns with atleast 1 missing value
MissingCount = Temp[Temp.DataMissing>0]

#sort 
MissingCount = MissingCount.sort_values('DataMissing',ascending=False)
#basic starts on missing data
print("##################################################")
print("###", '             Stats on missing data        ',  "###")
print("##################################################")
print('No of rows in HighschoolData: ',len(HighschoolData))
print('No of Columns in HighschoolData: ',len(HighschoolData.columns))
print('No. of Columns with Missing data: ',len(MissingCount))
print('No  of Complete data columns: ', len(HighschoolData.columns) - len(MissingCount))
print('No. of Columns with 100% missing Values: ',sum(1 for item in MissingCount.DataMissing if item==len(HighschoolData)))
print('No. of Columns more than 95% missing Values: ',sum(1 for item in MissingCount.DataMissing if item>=0.95*len(HighschoolData)))
print('% of Columns with some Missing data: ',round(float(100*len(MissingCount))/len(HighschoolData.columns),3),'%')
print('% of Columns with 100% Missing data: ',round(float(100*sum(1 for item in MissingCount.DataMissing if item==len(HighschoolData)))/len(HighschoolData.columns),3),'%')
print('% of Columns with more than 95% Missing data: ',round(float(100*sum(1 for item in MissingCount.DataMissing if item>=0.95*len(HighschoolData)))/len(HighschoolData.columns),3),'%')

print(" ")
print("##################################################")
print("###",'       Columns with >95% missing data     ',    "###")
print("##################################################")
#add a columnName for Bar charts plot
MissingCount['ColumnName'] = MissingCount.index
MissingCount['Missing %'] = 100*(MissingCount.DataMissing/len(HighschoolData))
MissingCount = MissingCount.reset_index()
del MissingCount['index']
print((MissingCount.head(100)))

##################################################
###              Stats on missing data         ###
##################################################
No of rows in HighschoolData:  478
No of Columns in HighschoolData:  294
No. of Columns with Missing data:  168
No  of Complete data columns:  126
No. of Columns with 100% missing Values:  47
No. of Columns more than 95% missing Values:  99
% of Columns with some Missing data:  57.143 %
% of Columns with 100% Missing data:  15.986 %
% of Columns with more than 95% Missing data:  33.673 %
 
##################################################
###        Columns with >95% missing data      ###
##################################################
    DataMissing                ColumnName   Missing %
0   478          Percent GLP               100.000000
1   478          total_expense_num         100.000000
2   478          GCE_RPF_State_Pct         100.000000
3   478          GCE_PRM_State_Pct         100.000000
4   478          GCE_LEP_State_

##### Approach for handling missing data to be discussed with the rest of the team
From above we note, of the 293 columns in HighSchoolData, 168 (57.14%) of them have some missing data. 47 (15.99%) of columns have all the data missing where 99 (33.67%) of the columns miss more than 95% of the data. For columns with 100% missing data we are left but to wonder if this were new fields introduced recently and the schools are not fully informed to collect this data or the data for not just collected by mistake. On the other hand, for the partally completed datasets, some schools migh have left this columns blank if they didn't apply to the.

For the sake of this analysis we will drop the columns that have `>95` missing data rather than try complete the missing data with either mean, median or mode. If we complete this large amount of columns, with predetermined data, our model maybe 

In [20]:
#All columns from Original schoolData
AllColumns =HighschoolData.columns 

# All columns with some missing value
ColumnsWithMissingdata = MissingCount.ColumnName 

#columns with more than 95% missing data
Columns2Drop = MissingCount.ColumnName.head(39) 

#columns with missing values that are kept
Columns2Keep = MissingCount.ColumnName.tail(len(MissingCount)-39) 

#All the other columns except those with >95% missing data
SelectedColumns = list(set(AllColumns)-set(Columns2Drop)) 

#new dataset, columns with >95% missing data dropped

schoolDataNew = HighschoolData[SelectedColumns]

## Peek at New Dataframe
print(HighschoolData)
print(schoolDataNew.info())
schoolDataNew.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 478 entries, 0 to 477
Columns: 255 entries, total_specialized_courses to PASSED_EOG_State_Pct
dtypes: bool(8), float64(225), int64(9), object(13)
memory usage: 926.2+ KB
None


total_specialized_courses  Math Score Reading SPG Grade  lea_wap_num  \
0 NaN                        NaN          NaN               1380.0        
1 NaN                        NaN          NaN               1380.0        

   st_flicensed_teach_pct  st_not_highqual_class_all_pct  \
0  0.897                   0.037                           
1  0.897                   0.037                           

   PASSED_EOG_School_Pct  tchyrs_0thru3_pct type_cd_txt  PASSED_LAA_Dist_Pct  \
0 NaN                     0.000              Public     NaN                    
1 NaN                     0.186              Public     NaN                    

           ...           Avg_Class_Size_05  _1yr_tchr_trnovr_pct  \
0          ...          NaN                 0.000                  
1          ...          NaN                 0.128                  

   st_highqual_class_hp_pct  Grad_project_status  Math Course Rigor Score  \
0  0.964                     False               NaN                        
1  0.964                     False               NaN                        

   lea_long_susp_per_c_num  Avg_Class_Size_04  st_highqual_class_pct  \
0  0.23                    NaN                 0.964                   
1  0.23                    NaN                 0.964                   

   Avg_Class_Size_03  PASSED_EOG_State_Pct  
0 NaN                NaN                    
1 NaN                NaN                    

[2 rows x 255 columns]

#### Simple Statistics [10]
Visualize appropriate statistics (e.g., range, mode, mean, median, variance, counts) for a subset of attributes. Describe anything meaningful you found from this or if you found something potentially interesting. Note: You can also use data from other sources for comparison. Explain why the statistics run are meaningful. 

#### Visualize Attributes [15]
Visualize the most interesting attributes (at least 5 attributes, your opinion on what is interesting). Important: Interpret the implications for each visualization. Explain for each attribute why the chosen visualization is appropriate.

#### Explore Joint Attributes [15]
Visualize relationships between attributes: Look at the attributes via scatter plots, correlation, cross-tabulation, group-wise averages, etc. as appropriate. Explain any interesting relationships.

#### New Features [5]
Are there other features that could be added to the data or created from existing features? Which ones?

#### Exceptional Work [10]
You have free reign to provide additional analyses. One idea: implement dimensionality reduction, then visualize and interpret the results.

#### 1. Data Dictionary
Since this datasets has numerous columns we needed a fast way to quickly find ColumnName description for easy reference. For exceptional work, we created a function to quickly pull the data from csv datafile. This involved converting the pdf to excel and formating the data for easy import into pandas. The code below is a working code for our data dictionary.

In [15]:
# This is a simple function to pull column description
DataDict = pd.read_csv(wd+'\\data\\dictionary.csv', encoding = "ISO-8859-1")
DataDict.head()
#DataDict = DataDict.columns['COLUMN_NAME', 'DESCRIPTION']
def get_ColDescription(colname = 'Year'):
    colName = input("Enter column name to check description in Dictionary. You can enter multiple columns separated by comma: ")
    
    print('You entered: ', colName.strip())
    temp = pd.DataFrame()
    colNames = colName.split(',')
    
    try:
        for i in range(0,len(colNames)):
            get = (DataDict[DataDict.COLUMN_NAME==colNames[i].strip().lower()])
            temp = temp.append(get)
        return(temp)
    except Exception as e:
        print(e.args) 

get_ColDescription()

Enter column name to check description in Dictionary. You can enter multiple columns separated by comma: A, T, M, I, E
You entered:  A, T, M, I, E


COLUMN_NAME                                            DESCRIPTION
373  a           School has elementary, middle, and high school grades
372  t           School has middle and high school grades             
369  m           School has middle schools grades (6-8)               
371  i           School has elementary and middle school grades       
368  e           School has elementary schools grades (PK-5)